# Лабораторная работа No1
Имеется следующие оперативные данные по датчикам телеметрии `SensorData.csv`  
Пример одной строки:
```
1;1666200364421000;16882.26501
```
где:
```
1 - номер датчика;
1666200364421000 - дата в UNIX_TIMESTAMP
16882.26501 - значение датчика;
```
В данных возможны обновления записей (UPDATE) на основании номера
датчика  
Имеется справочник датчиков телеметрии `SensorInfo.xlsx (SensorInfoDict.csv)`
1) Создайте модель данных хранения информации по предоставленным
данным.
2) Укажите какие параметры хранения по вашему мнению необходимы для
той или иной таблицы
- FILLFACTOR
- Политика работы с TOAST таблицами
3) Создайте виртуальную таблицу (VIEW) в вашей схеме базы данных для
предоставления данных вида
- Название датчика;
- Дата значения датчика в формате DD.MM.YYYY HH24:MI:SS
- Значение датчика с округлением до 2 знаков после запятой

## Создаем подключение к БД

In [125]:
import psycopg2
### Подключение к PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="",
    user="",
    password="",
)
### Создание курсора
cursor = connection.cursor()

print("Подключение к PostgreSQL успешно.")

Подключение к PostgreSQL успешно.


## Создание схемы

In [126]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS ivan_patakin;")
connection.commit()

print("Схема 'ivan_patakin' создана.")

Схема 'ivan_patakin' создана.


## Создание таблиц
### Создадим справочники для хранения "Тип значения" и "Единица измерения" 

In [127]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS ivan_patakin.value_type_dict (
    id SERIAL PRIMARY KEY,
    value_type VARCHAR(50) NOT NULL
);
""")
connection.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS ivan_patakin.unit_dict (
    id SERIAL PRIMARY KEY,
    unit VARCHAR(50) NOT NULL
);
""")
connection.commit()

print("Справочники value_type_dict и unit_dict созданы")

Справочники value_type_dict и unit_dict созданы


Установим FILLFACTOR для справочников на 100%
- Справочники будут использоваться для хранения небольшого количества строк, и их размер будет небольшим.
- Установка FILLFACTOR на 100% позволит избежать дополнительных затрат на хранение и ускорить доступ к данным.

In [128]:
cursor.execute("""
ALTER TABLE ivan_patakin.value_type_dict SET (FILLFACTOR = 100);
ALTER TABLE ivan_patakin.unit_dict SET (FILLFACTOR = 100);
""")
connection.commit()

print("FILLFACTOR для справочников установлен на 100%")

FILLFACTOR для справочников установлен на 100%


#### Заполним справочники данными

In [129]:
cursor.execute("""
               INSERT INTO ivan_patakin.value_type_dict (value_type) VALUES ('PV'), ('MV');
                """)
connection.commit()

cursor.execute("""
INSERT INTO ivan_patakin.unit_dict (unit) VALUES ('кг/ч'), ('КПа'), ('град. C'), ('%'), ('дискретный'), ('% масс.');
""")
connection.commit()

print("Справочники value_type_dict и unit_dict заполнены")

Справочники value_type_dict и unit_dict заполнены


### Создадим справочник датчиков телеметрии и таблицу для данных с датчиков

In [130]:
cursor.execute("""
CREATE TABLE ivan_patakin.sensor_info_dict (
    sensor_id INT PRIMARY KEY NOT NULL,
    sensor_name VARCHAR(100) NOT NULL,
    description TEXT,
    value_type_id INT REFERENCES ivan_patakin.value_type_dict(id) ON DELETE CASCADE,
    unit_id INT REFERENCES ivan_patakin.unit_dict(id) ON DELETE CASCADE,
    min_value FLOAT,
    max_value FLOAT
);
""")
connection.commit()

print("Справочник sensor_info_dict создана")

Справочник sensor_info_dict создана


In [131]:
cursor.execute("""
CREATE TABLE ivan_patakin.sensor_data (
    sensor_id INT REFERENCES ivan_patakin.sensor_info_dict(sensor_id) ON DELETE CASCADE NOT NULL,
    unix_timestamp BIGINT NOT NULL,
    sensor_value FLOAT NOT NULL
    );
               """)
connection.commit()
print("Таблица sensor_data создана")

Таблица sensor_data создана


Установим FILLFACTOR для таблиц sensor_info_dict и sensor_data.
- Таблица sensor_info_dict будет содержать информацию о датчиках, которая может изменяться со временем. Установка FILLFACTOR на 90% позволит уменьшить фрагментацию и ускорить операции вставки и обновления.
- Таблица sensor_data будет содержать большие объемы данных, которые будут записываться в нее регулярно. Установка FILLFACTOR на 80% позволит уменьшить фрагментацию данных при частых обновлениях и обеспечить более эффективное использование памяти.

In [132]:
cursor.execute("""
ALTER TABLE ivan_patakin.sensor_info_dict SET (FILLFACTOR = 90);
ALTER TABLE ivan_patakin.sensor_data SET (FILLFACTOR = 80);
""")
connection.commit()

print("FILLFACTOR для справочника sensor_info_dict и таблицы sensor_data установлен")

FILLFACTOR для справочника sensor_info_dict и таблицы sensor_data установлен


Политика работы с TOAST таблицами
- Для таблицы sensor_info_dict установим политику работы с параметром description на EXTERNAL, так как это поле может содержать большие объемы данных (например, описание датчика), и мы хотим хранить его в отдельной TOAST таблице.

In [133]:
cursor.execute("ALTER TABLE ivan_patakin.sensor_info_dict ALTER COLUMN description SET STORAGE EXTERNAL;")

print("Политика работы с TOAST таблицей для справочника sensor_info_dict установлена на EXTERNAL")

Политика работы с TOAST таблицей для справочника sensor_info_dict установлена на EXTERNAL


#### Заполним справочник sensor_info_dict данными из SensorInfoDict.csv

In [134]:
import csv
with open('Data/Lab1/SensorInfoDict.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        cursor.execute("SELECT id FROM ivan_patakin.value_type_dict WHERE value_type = %s;", (row['Тип значения'],))
        value_type_id = cursor.fetchone()[0]
        
        cursor.execute("SELECT id FROM ivan_patakin.unit_dict WHERE unit = %s;", (row['Единица измерения'],))
        unit_id = cursor.fetchone()[0]
        
        cursor.execute("""
            INSERT INTO ivan_patakin.sensor_info_dict (sensor_id, sensor_name, description, value_type_id, unit_id, min_value, max_value)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (row['Идентификатор'], row['Название датчика'], row['Описание'], value_type_id, unit_id, float(row['min'].replace(',','.')), float(row['max'].replace(',','.'))))
connection.commit()

print("Справочник sensor_info_dict заполнен")

Справочник sensor_info_dict заполнен


#### Заполним таблицу sensor_data данными из SensorData.csv

In [135]:
with open('Data/Lab1/SensorData.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        cursor.execute("""
            INSERT INTO ivan_patakin.sensor_data (sensor_id, unix_timestamp, sensor_value)
            VALUES (%s, %s, %s);
        """, (row[0], row[1], float(row[2].replace(',','.'))))
connection.commit()

print("Таблица sensor_data заполнена")

Таблица sensor_data заполнена


## Создадим VIEW для просмотра загруженных данных

In [136]:
cursor.execute("""
CREATE OR REPLACE VIEW ivan_patakin.sensor_data_view AS
SELECT
    sid.sensor_name AS "Название датчика",
    to_timestamp(sd.unix_timestamp / 1000000)::timestamp AS "Дата значения датчика",
    ROUND(CAST(sd.sensor_value AS NUMERIC), 2) AS "Значение датчика"
FROM
    ivan_patakin.sensor_data sd
JOIN
    ivan_patakin.sensor_info_dict sid ON sd.sensor_id = sid.sensor_id;
""")

connection.commit()

print("Виртуальная таблица sensor_data_view создана")

Виртуальная таблица sensor_data_view создана


## Выведем данные в sensor_data_view
```sql
select * from ivan_patakin.sensor_data_view
```

## Удалим таблицы и справочники

In [137]:
cursor.execute("""
DROP VIEW IF EXISTS ivan_patakin.sensor_data_view;
DROP TABLE IF EXISTS ivan_patakin.sensor_data;
DROP TABLE IF EXISTS ivan_patakin.sensor_info_dict;
DROP TABLE IF EXISTS ivan_patakin.unit_dict;
DROP TABLE IF EXISTS ivan_patakin.value_type_dict;
                """)
connection.commit()

print("Таблицы удалены")

Таблицы удалены


In [138]:
### Закрытие курсора и соединения
cursor.close()
connection.close()

print("Соединение с PostgreSQL закрыто.")

Соединение с PostgreSQL закрыто.
